In [141]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/south-african-language-identification-hack-2022/sample_submission.csv
/kaggle/input/south-african-language-identification-hack-2022/test_set.csv
/kaggle/input/south-african-language-identification-hack-2022/train_set.csv


- This is a baseline model created on kaggle, 
# MEANT TO BE IMPROVED UPON!!!

In [142]:
import collections # used for dictionaries and counters
from itertools import permutations # used to find permutations

In [143]:
train = pd.read_csv('/kaggle/input/south-african-language-identification-hack-2022/train_set.csv')
test = pd.read_csv('/kaggle/input/south-african-language-identification-hack-2022/test_set.csv')
ss = pd.read_csv('/kaggle/input/south-african-language-identification-hack-2022/sample_submission.csv')

In [144]:
train.head()

,lang_id,text
0,xho,umgaqo-siseko wenza amalungiselelo kumaziko ax...
1,xho,i-dha iya kuba nobulumko bokubeka umsebenzi na...
2,eng,the province of kwazulu-natal department of tr...
3,nso,o netefatša gore o ba file dilo ka moka tše le...
4,ven,khomishini ya ndinganyiso ya mbeu yo ewa maana...


In [145]:
test.head()

,index,text
0,1,"Mmasepala, fa maemo a a kgethegileng a letlele..."
1,2,Uzakwaziswa ngokufaneleko nakungafuneka eminye...
2,3,Tshivhumbeo tshi fana na ngano dza vhathu.
3,4,Kube inja nelikati betingevakala kutsi titsini...
4,5,Winste op buitelandse valuta.


- converting the values from object to strings

In [146]:
def trans_data(dataset):
    # Define a list of commonly found punctuations
    punc = ('!', "," ,"\'" ,";" ,"\"", ".", "-" ,"?")
    vowels=['a','e','i','o','u']
    # Define a list of double consecutive vowels which are typically found in Dutch and Afrikaans languages
    same_consecutive_vowels = ['aa','ee', 'ii', 'oo', 'uu'] 
    consecutive_vowels = [''.join(p) for p in permutations(vowels,2)]
    dutch_combos = ['ij']
    data = dataset.copy()
    data['text'] = data['text'].astype(str)
    # Create a pre-defined set of features based on the "text" column in order to allow us to characterize the string
    data['word_count'] = data['text'].apply(lambda x : len(x.split()))
    data['character_count'] = data['text'].apply(lambda x : len(x.replace(" ","")))
    data['word_density'] = data['word_count'] / (data['character_count'] + 1)
    data['punc_count'] = data['text'].apply(lambda x : len([a for a in x if a in punc]))
    data['v_char_count'] = data['text'].apply(lambda x : len([a for a in x if a.casefold() == 'v']))
    data['w_char_count'] = data['text'].apply(lambda x : len([a for a in x if a.casefold() == 'w']))
    data['ij_char_count'] = data['text'].apply(lambda x : sum([any(d_c in a for d_c in dutch_combos) for a in x.split()]))
    data['num_double_consec_vowels'] = data['text'].apply(lambda x : sum([any(c_v in a for c_v in same_consecutive_vowels) for a in x.split()]))
    data['num_consec_vowels'] = data['text'].apply(lambda x : sum([any(c_v in a for c_v in consecutive_vowels) for a in x.split()]))
    data['num_vowels'] = data['text'].apply(lambda x : sum([any(v in a for v in vowels) for a in x.split()]))
    data['vowel_density'] = data['num_vowels']/data['word_count']
    data['capitals'] = data['text'].apply(lambda comment: sum(1 for c in comment if c.isupper()))
    data['caps_vs_length'] = data.apply(lambda row: float(row['capitals'])/float(row['character_count']),axis=1)
    data['num_exclamation_marks'] =data['text'].apply(lambda x: x.count('!'))
    data['num_question_marks'] = data['text'].apply(lambda x: x.count('?'))
    data['num_punctuation'] = data['text'].apply(lambda x: sum(x.count(w) for w in punc))
    data['num_unique_words'] = data['text'].apply(lambda x: len(set(w for w in x.split())))
    data['num_repeated_words'] = data['text'].apply(lambda x: len([w for w in collections.Counter(x.split()).values() if w > 1]))
    data['words_vs_unique'] = data['num_unique_words'] / data['word_count']
    data['encode_ascii'] = np.nan
    for i in range(len(data)):
        try:
            data['text'].iloc[i].encode(encoding='utf-8').decode('ascii')
        except UnicodeDecodeError:
            data['encode_ascii'].iloc[i] = 0
        else:
            data['encode_ascii'].iloc[i] = 1
            
    return data

In [147]:
%%time
train = trans_data(train)
test = trans_data(test)

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


CPU times: user 29.4 s, sys: 11.4 ms, total: 29.5 s
Wall time: 29.6 s


In [148]:
train.head()

,lang_id,text,word_count,character_count,word_density,punc_count,v_char_count,w_char_count,ij_char_count,num_double_consec_vowels,...,vowel_density,capitals,caps_vs_length,num_exclamation_marks,num_question_marks,num_punctuation,num_unique_words,num_repeated_words,words_vs_unique,encode_ascii
0,xho,umgaqo-siseko wenza amalungiselelo kumaziko ax...,24,197,0.121212,1,0,6,0,0,...,1.000,0,0.0,0,0,1,23,1,0.958333,1.0
1,xho,i-dha iya kuba nobulumko bokubeka umsebenzi na...,31,222,0.139013,2,1,4,0,1,...,1.000,0,0.0,0,0,2,30,1,0.967742,1.0
2,eng,the province of kwazulu-natal department of tr...,37,228,0.161572,1,3,4,0,0,...,1.000,0,0.0,0,0,1,27,5,0.729730,1.0
3,nso,o netefatša gore o ba file dilo ka moka tše le...,40,178,0.223464,0,0,1,0,0,...,0.975,0,0.0,0,0,0,31,5,0.775000,0.0
4,ven,khomishini ya ndinganyiso ya mbeu yo ewa maana...,43,197,0.217172,0,1,3,0,1,...,1.000,0,0.0,0,0,0,24,8,0.558140,1.0


In [149]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
train['lang_id'] = le.fit_transform(train["lang_id"])

In [151]:
target = train.pop('lang_id')
text = train.pop('text')

test_text = test.pop('text')

In [163]:
test_index = test.pop('index')

In [153]:
from sklearn.model_selection import train_test_split

#split dataset into features and target variable
# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(train, target, test_size=0.2) # 80% train and 20% test

In [154]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
import lightgbm

In [155]:
dt_clf = DecisionTreeClassifier() # Create Decision Tree classifer object
dt_clf = dt_clf.fit(X_train,y_train) # Fit/Train Decision Tree Classifer on training set

# Save model to file in the current working directory so that it can be imported and used.
# I use the pickle library to save the parameters of the trained model
# pkl_file = "decision_tree_model.pkl"
# with open(pkl_file, 'wb') as file:
#     pickle.dump(dt_clf, file)

# # Load previously trained model from pickle file
# with open(pkl_file, 'rb') as file:
#     dt_clf = pickle.load(file)

# dt_clf # parameters of the Decision Tree model are shown below and can be further optimized to improve model performance

y_pred = dt_clf.predict(X_test) #Predict the response for test dataset

In [156]:
from sklearn.metrics import accuracy_score

In [157]:
# Calculate accuracy of the Decision Tree Model
accuracy_score_dt = accuracy_score(y_test, y_pred)
accuracy_score_dt

0.6628787878787878

In [158]:
z = lightgbm.LGBMClassifier()
z.fit(X_train,y_train)
accuracy_score(z.predict(X_test) , y_test)

0.708030303030303

In [159]:
rf_clf = RandomForestClassifier(n_estimators=100) # Create Random Forest classifer object
rf_clf = rf_clf.fit(X_train,y_train) # Fit/Train Random Forest Classifer on training set

#Predict the response for test dataset
accuracy_score(y_test , rf_clf.predict(X_test))

0.7222727272727273

In [166]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

trans_train = sc.fit_transform(train)
trans_test = sc.fit_transform(test)

In [167]:
rf_clf.predict(trans_test)

/opt/conda/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  "X does not have valid feature names, but"


array([3, 5, 3, ..., 3, 3, 5])

In [169]:
preds = np.vstack(le.inverse_transform(rf_clf.predict(test)))

array([['sot'],
       ['ssw'],
       ['tso'],
       ...,
       ['tsn'],
       ['tsn'],
       ['zul']], dtype='<U3')

In [177]:
pd.DataFrame(zed).to_csv('preds.csv' , index=False)

,Index,Target
0,1,sot
1,2,ssw
2,3,tso
3,4,zul
4,5,zul
...,...,...
5677,5678,sot
5678,5679,nso
5679,5680,tsn
5680,5681,tsn


In [178]:
ss

,index,lang_id
0,1,tsn
1,2,nbl
